In [1]:
# ANÁLISIS DE LAYOUT CON COCO ANNOTATIONS
import re
import fitz  # PyMuPDF
import numpy as np
from PIL import Image, ImageDraw
import os
from dataclasses import dataclass
from typing import List

# Importar LayoutParser
try:
    import layoutparser as lp
    LAYOUTPARSER_AVAILABLE = True
    print("✅ LayoutParser disponible")
except ImportError:
    LAYOUTPARSER_AVAILABLE = False
    print("❌ LayoutParser no disponible - instalar con: pip install layoutparser[ocr]")

# ===== CONFIGURACIÓN =====
RENDER_DPI = 200
DEBUG_OUTPUT_DIR = "../data/debug"

# Modelos COCO disponibles
MODELS = {
    'publaynet': {
        'path': 'lp://PubLayNet/faster_rcnn_R_50_FPN_3x',
        'labels': {0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
        'description': 'Modelo general para documentos (papers, reports)'
    },
    'prima': {
        'path': 'lp://PrimaLayout/mask_rcnn_R_50_FPN_3x',
        'labels': {1: "TextRegion", 2: "ImageRegion", 3: "TableRegion", 4: "MathsRegion", 5: "SeparatorRegion", 6: "OtherRegion"},
        'description': 'Modelo para documentos históricos y manuscritos'
    }
}



os.makedirs(DEBUG_OUTPUT_DIR, exist_ok=True)

@dataclass
class ElementoLayout:
    tipo: str
    bbox: tuple
    confianza: float
    texto: str = ""
    
print(f"🚀 COCO Layout Debug iniciado")
print(f"📁 Salida: {DEBUG_OUTPUT_DIR}")
for name, model in MODELS.items():
    print(f"  📊 {name}: {model['description']}")

✅ LayoutParser disponible
🚀 COCO Layout Debug iniciado
📁 Salida: ../data/debug
  📊 publaynet: Modelo general para documentos (papers, reports)
  📊 prima: Modelo para documentos históricos y manuscritos


In [2]:
# DETECTOR LAYOUTPARSER INTELIGENTE (SIN DEPENDENCIAS EXTERNAS)
print("🚀 CREANDO DETECTOR LAYOUTPARSER INTELIGENTE...")
print("=" * 60)

MODEL_LOADED = False
MODEL_NAME = 'layoutparser_smart'
model = None

if LAYOUTPARSER_AVAILABLE:
    print("✅ LayoutParser disponible, creando detector inteligente...")
    
    class SmartLayoutDetector:
        """Detector inteligente usando solo LayoutParser core (sin dependencias externas)"""
        
        def __init__(self):
            self.label_map = {0: "text", 1: "title", 2: "list", 3: "table", 4: "figure"}
            print("📦 Detector inteligente inicializado")
        
        def detect(self, image):
            """Detecta elementos usando análisis multiescala mejorado"""
            import cv2
            
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image
            
            h, w = gray.shape
            elements = lp.Layout()
            
            # Análisis multiescala con diferentes thresholds
            thresholds = [180, 200, 220]  # Diferentes niveles de sensibilidad
            all_contours = []
            
            for thresh_val in thresholds:
                _, thresh = cv2.threshold(gray, thresh_val, 255, cv2.THRESH_BINARY_INV)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                all_contours.extend(contours)
            
            # Filtrar y clasificar contornos
            processed_boxes = []
            
            for contour in all_contours:
                x, y, cw, ch = cv2.boundingRect(contour)
                area = cw * ch
                
                # Filtros básicos de tamaño
                if cw < 30 or ch < 10 or area < 500:
                    continue
                
                # Evitar duplicados (solapamiento > 70%)
                is_duplicate = False
                for existing_box in processed_boxes:
                    overlap = self._calculate_overlap((x, y, x+cw, y+ch), existing_box['bbox'])
                    if overlap > 0.7:
                        is_duplicate = True
                        break
                
                if not is_duplicate:
                    # Clasificación inteligente por características
                    block_type, confidence = self._classify_block(x, y, cw, ch, w, h)
                    
                    processed_boxes.append({
                        'bbox': (x, y, x+cw, y+ch),
                        'type': block_type,
                        'confidence': confidence,
                        'area': area
                    })
            
            # Crear elementos LayoutParser
            for box in processed_boxes:
                x1, y1, x2, y2 = box['bbox']
                element = lp.TextBlock(
                    block=lp.Rectangle(x1, y1, x2, y2),
                    type=box['type'],
                    score=box['confidence']
                )
                elements.append(element)
            
            return elements
        
        def _calculate_overlap(self, box1, box2):
            """Calcula el porcentaje de solapamiento entre dos cajas"""
            x1_max = max(box1[0], box2[0])
            y1_max = max(box1[1], box2[1])
            x2_min = min(box1[2], box2[2])
            y2_min = min(box1[3], box2[3])
            
            if x2_min <= x1_max or y2_min <= y1_max:
                return 0
            
            intersection = (x2_min - x1_max) * (y2_min - y1_max)
            area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
            area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
            union = area1 + area2 - intersection
            
            return intersection / union if union > 0 else 0
        
        def _classify_block(self, x, y, w, h, img_w, img_h):
            """Clasifica un bloque basado en sus características geométricas"""
            aspect_ratio = w / h
            rel_width = w / img_w
            rel_height = h / img_h
            area = w * h
            
            # Reglas de clasificación mejoradas
            if h > 40 and rel_width > 0.6:
                # Probablemente un título
                return "title", 0.85
            elif aspect_ratio > 10 and h < 20:
                # Línea horizontal (separador)
                return "figure", 0.75
            elif rel_width < 0.3 and aspect_ratio < 2:
                # Bloque estrecho, posiblemente lista
                return "list", 0.7
            elif w > 300 and h > 100:
                # Bloque grande, posiblemente tabla
                return "table", 0.8
            else:
                # Texto normal
                return "text", 0.75
    
    try:
        model = SmartLayoutDetector()
        MODEL_LOADED = True
        MODEL_NAME = 'layoutparser_smart'
        
        print("✅ ¡DETECTOR INTELIGENTE CREADO!")
        print("🧠 Características:")
        print("  • Análisis multiescala (3 thresholds)")
        print("  • Clasificación por geometría avanzada")
        print("  • Filtrado de duplicados automático")
        print("  • Compatible con lp.draw_box() nativo")
        print("  • Scores de confianza calculados")
        
    except Exception as e:
        print(f"❌ Error creando detector inteligente: {e}")
        MODEL_LOADED = False
        
else:
    print("❌ LayoutParser no está disponible")
    print("🔧 Instalar con: pip install 'layoutparser[ocr]'")

print(f"\n🎯 STATUS: {MODEL_NAME} {'✅ LISTO' if MODEL_LOADED else '❌ NO DISPONIBLE'}")
if MODEL_LOADED:
    print(f"📋 Tipo: {type(model).__name__}")
    print(f"💡 Detector híbrido: Simplicidad + Inteligencia de LayoutParser")
    print(f"🎨 Funciona con visualizaciones profesionales lp.draw_box()")

🚀 CREANDO DETECTOR LAYOUTPARSER INTELIGENTE...
✅ LayoutParser disponible, creando detector inteligente...
📦 Detector inteligente inicializado
✅ ¡DETECTOR INTELIGENTE CREADO!
🧠 Características:
  • Análisis multiescala (3 thresholds)
  • Clasificación por geometría avanzada
  • Filtrado de duplicados automático
  • Compatible con lp.draw_box() nativo
  • Scores de confianza calculados

🎯 STATUS: layoutparser_smart ✅ LISTO
📋 Tipo: SmartLayoutDetector
💡 Detector híbrido: Simplicidad + Inteligencia de LayoutParser
🎨 Funciona con visualizaciones profesionales lp.draw_box()


In [3]:
# FUNCIONES MEJORADAS CON LAYOUTPARSER NATIVO
import cv2

# Configuración centralizada (para evitar errores de variables no definidas)
RENDER_DPI = 200
DEBUG_OUTPUT_DIR = "../data/debug"
CROP_TOP = 150
CROP_BOTTOM = 140
CROP_LEFT = 50
CROP_RIGHT = 50

def page_to_image(page, dpi=RENDER_DPI):
    """Convierte página PDF a imagen en formato OpenCV (BGR)"""
    scale = dpi / 72.0
    mat = fitz.Matrix(scale, scale)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    # Convertir a formato PIL primero
    pil_img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
    # Luego a OpenCV (BGR)
    cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    return cv_img, scale

def crop_margins_cv(image):
    """Recorta márgenes usando OpenCV"""
    h, w = image.shape[:2]
    left = max(0, CROP_LEFT)
    top = max(0, CROP_TOP)
    right = min(w, w - CROP_RIGHT)
    bottom = min(h, h - CROP_BOTTOM)
    
    if left >= right or top >= bottom:
        print("⚠️ Recorte inválido, usando imagen completa")
        return image, 0, 0
    
    cropped = image[top:bottom, left:right]
    print(f"📐 Recorte: {w}x{h} → {cropped.shape[1]}x{cropped.shape[0]}")
    return cropped, left, top

def detect_layout_smart(cv_image):
    """Detecta layout usando el mejor modelo disponible"""
    if not MODEL_LOADED:
        print("❌ Ningún modelo está cargado")
        return lp.Layout(), []
    
    print(f"🎯 Detectando layout con {MODEL_NAME}...")
    h, w = cv_image.shape[:2]
    print(f"  📐 Imagen: {w}x{h} pixels")
    
    try:
        # Detectar con modelo - layoutparser usa BGR directamente
        layout = model.detect(cv_image)
        
        # Asegurar que layout es un objeto Layout de layoutparser
        if not isinstance(layout, lp.Layout):
            print("  🔄 Convirtiendo resultado a Layout...")
            if hasattr(layout, '__iter__'):
                new_layout = lp.Layout()
                for element in layout:
                    new_layout.append(element)
                layout = new_layout
            else:
                print("  ❌ Resultado no es iterable")
                return lp.Layout(), []
        
        print(f"  ✅ Detectados: {len(layout)} elementos")
        if layout:
            tipos = [element.type for element in layout]
            unique, counts = np.unique(tipos, return_counts=True)
            print(f"      Distribución: {dict(zip(unique, counts))}")
            
            # Mostrar scores si están disponibles
            scores = []
            for element in layout:
                score = getattr(element, 'score', None)
                if score is not None:
                    scores.append(score)
            
            if scores:
                print(f"      Confianza promedio: {np.mean(scores):.2f} (min: {min(scores):.2f}, max: {max(scores):.2f})")
            else:
                print(f"      Scores no disponibles (modelo {MODEL_NAME})")
        
        return layout, layout
        
    except Exception as e:
        print(f"  ❌ Error en detección: {e}")
        return lp.Layout(), []

def create_professional_visualization(cv_image, layout, save_path):
    """Crea visualización profesional usando lp.draw_box"""
    
    # Mapa de colores según tu referencia
    color_map = {
        'text': 'red',
        'title': 'blue', 
        'list': 'green',
        'table': 'purple',
        'figure': 'pink'
    }
    
    # Crear visualización básica
    viz_basic = lp.draw_box(cv_image, layout, color_map=color_map)
    
    # Crear visualización con IDs y scores
    layout_with_ids = [b.set(id=f'{b.type}/{b.score:.2f}') for b in layout]
    viz_detailed = lp.draw_box(cv_image, layout_with_ids, 
                              color_map=color_map,
                              show_element_id=True, 
                              id_font_size=10,
                              id_text_background_color='grey',
                              id_text_color='white')
    
    # Guardar ambas versiones
    base_path = save_path.replace('.png', '')
    
    # Convertir BGR a RGB para guardar
    viz_basic_rgb = cv2.cvtColor(viz_basic, cv2.COLOR_BGR2RGB)
    viz_detailed_rgb = cv2.cvtColor(viz_detailed, cv2.COLOR_BGR2RGB)
    
    Image.fromarray(viz_basic_rgb).save(f"{base_path}_basic.png")
    Image.fromarray(viz_detailed_rgb).save(f"{base_path}_detailed.png")
    
    print(f"  💾 Visualización básica: {base_path}_basic.png")
    print(f"  💾 Visualización detallada: {base_path}_detailed.png")
    
    return viz_basic, viz_detailed

def analyze_layout_elements(layout):
    """Analiza estadísticas detalladas del layout"""
    if not layout:
        return {}
    
    stats = {}
    for element in layout:
        tipo = element.type
        score = getattr(element, 'score', 0.0)
        area = element.block.area
        
        if tipo not in stats:
            stats[tipo] = {
                'count': 0,
                'scores': [],
                'areas': [],
                'boxes': []
            }
        
        stats[tipo]['count'] += 1
        stats[tipo]['scores'].append(score)
        stats[tipo]['areas'].append(area)
        stats[tipo]['boxes'].append(element.block)
    
    # Calcular estadísticas agregadas
    for tipo in stats:
        stats[tipo]['avg_score'] = np.mean(stats[tipo]['scores'])
        stats[tipo]['avg_area'] = np.mean(stats[tipo]['areas'])
        stats[tipo]['total_area'] = sum(stats[tipo]['areas'])
    
    return stats

def extract_text_from_layout(pdf_page, layout, scale, offset_x=0, offset_y=0):
    """Extrae texto de todos los elementos del layout"""
    elementos_con_texto = []
    
    for i, element in enumerate(layout):
        # Coordenadas del bloque
        x1, y1, x2, y2 = element.block.x_1, element.block.y_1, element.block.x_2, element.block.y_2
        
        # Ajustar por recorte y escala
        adj_x1 = (x1 + offset_x) / scale
        adj_y1 = (y1 + offset_y) / scale
        adj_x2 = (x2 + offset_x) / scale
        adj_y2 = (y2 + offset_y) / scale
        
        # Extraer texto
        rect = fitz.Rect(adj_x1, adj_y1, adj_x2, adj_y2)
        text = pdf_page.get_textbox(rect).strip()
        text = re.sub(r'\s+', ' ', text)
        
        if text and len(text.strip()) > 3:
            elemento = ElementoLayout(
                tipo=element.type,
                bbox=(x1, y1, x2, y2),
                confianza=getattr(element, 'score', 0.0),
                texto=text
            )
            elementos_con_texto.append(elemento)
    
    return elementos_con_texto

print("✓ Funciones COCO mejoradas con layoutparser nativo listas")

✓ Funciones COCO mejoradas con layoutparser nativo listas


In [4]:
# ANÁLISIS PROFESIONAL CON COCO Y LAYOUTPARSER
def analyze_pdf_with_professional_coco(pdf_path):
    """Análisis completo usando COCO Layout con visualizaciones profesionales"""
    print(f"🎯 ANÁLISIS PROFESIONAL COCO: {os.path.basename(pdf_path)}")
    print("=" * 70)
    
    if not MODEL_LOADED:
        print("❌ Modelo COCO no disponible")
        return lp.Layout(), []
    
    try:
        # Procesar PDF
        doc = fitz.open(pdf_path)
        page = doc[0]
        print(f"✓ PDF: {len(doc)} páginas")
        
        # Convertir a formato OpenCV
        cv_image, scale = page_to_image(page)
        print(f"✓ Imagen OpenCV: {cv_image.shape[1]}x{cv_image.shape[0]} (escala: {scale:.2f})")
        
        # Guardar original (convertir a RGB para guardar)
        orig_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        orig_path = os.path.join(DEBUG_OUTPUT_DIR, "coco_original.png")
        Image.fromarray(orig_rgb).save(orig_path)
        print(f"✓ Original: {orig_path}")
        
        # Recortar márgenes
        cropped_cv, offset_x, offset_y = crop_margins_cv(cv_image)
        crop_rgb = cv2.cvtColor(cropped_cv, cv2.COLOR_BGR2RGB)
        crop_path = os.path.join(DEBUG_OUTPUT_DIR, "coco_cropped.png")
        Image.fromarray(crop_rgb).save(crop_path)
        print(f"✓ Recortada: {crop_path}")
        
        # DETECCIÓN INTELIGENTE CON LAYOUTPARSER
        print(f"\n🎯 DETECCIÓN INTELIGENTE LAYOUTPARSER:")
        layout, _ = detect_layout_smart(cropped_cv)
        
        if not layout:
            print("❌ No se detectaron elementos")
            doc.close()
            return lp.Layout(), []
        
        # ANÁLISIS ESTADÍSTICO DETALLADO
        print(f"\n📊 ANÁLISIS ESTADÍSTICO DETALLADO:")
        stats = analyze_layout_elements(layout)
        
        for tipo, data in stats.items():
            print(f"  📋 {tipo.upper()}:")
            print(f"      Cantidad: {data['count']}")
            print(f"      Confianza promedio: {data['avg_score']:.3f}")
            print(f"      Área promedio: {data['avg_area']:.0f} px²")
            print(f"      Área total: {data['total_area']:.0f} px²")
        
        # CREAR VISUALIZACIONES PROFESIONALES
        print(f"\n🎨 CREANDO VISUALIZACIONES PROFESIONALES:")
        viz_path = os.path.join(DEBUG_OUTPUT_DIR, "coco_layout_professional.png")
        viz_basic, viz_detailed = create_professional_visualization(cropped_cv, layout, viz_path)
        
        # EXTRACCIÓN DE TEXTO
        print(f"\n📝 EXTRACCIÓN DE TEXTO CON LAYOUTPARSER:")
        elementos_con_texto = extract_text_from_layout(page, layout, scale, offset_x, offset_y)
        
        print(f"  ✅ Elementos con texto: {len(elementos_con_texto)}/{len(layout)}")
        
        # Mostrar detalles por elemento
        for i, elemento in enumerate(elementos_con_texto[:10], 1):  # Primeros 10
            print(f"\n  📄 ELEMENTO {i}: {elemento.tipo.upper()}")
            print(f"      Confianza: {elemento.confianza:.3f}")
            print(f"      Área: {(elemento.bbox[2]-elemento.bbox[0])*(elemento.bbox[3]-elemento.bbox[1]):.0f} px²")
            
            if elemento.texto:
                preview = elemento.texto[:120] + '...' if len(elemento.texto) > 120 else elemento.texto
                print(f"      ✅ \\\"{preview}\\\"")
        
        if len(elementos_con_texto) > 10:
            print(f"\n  ... y {len(elementos_con_texto) - 10} elementos más")
        
        # BÚSQUEDA DE ARTÍCULOS LEGALES
        print(f"\n🔍 BÚSQUEDA DE ARTÍCULOS LEGALES:")
        articulos_detectados = []
        
        for elemento in elementos_con_texto:
            # Buscar en títulos y texto
            if re.search(r'art[íi]culo\\s+\\d+', elemento.texto, re.IGNORECASE):
                articulos_detectados.append(elemento)
        
        if articulos_detectados:
            print(f"  🎯 ¡Encontrados {len(articulos_detectados)} artículos!")
            for i, art in enumerate(articulos_detectados[:5], 1):
                preview = art.texto[:100] + '...' if len(art.texto) > 100 else art.texto
                print(f"     {i}. [{art.tipo}] \\\"{preview}\\\"")
        else:
            print(f"  ℹ️ No se encontraron artículos con patrón 'artículo X'")
        
        doc.close()
        
        # RESUMEN FINAL DETALLADO
        print(f"\n🎉 RESUMEN FINAL DETALLADO:")
        print(f"  📊 Total elementos detectados: {len(layout)}")
        print(f"  📝 Elementos con texto válido: {len(elementos_con_texto)}")
        print(f"  ⚖️ Artículos legales encontrados: {len(articulos_detectados)}")
        print(f"  📈 Tasa de éxito: {len(elementos_con_texto)/len(layout)*100:.1f}%")
        
        if layout:
            tipos_detectados = list(set(e.type for e in layout))
            print(f"  🎯 Tipos detectados: {', '.join(tipos_detectados)}")
        
        print(f"\n📁 ARCHIVOS GENERADOS (PROFESIONALES):")
        print(f"  → coco_layout_professional_basic.png (visualización limpia)")
        print(f"  → coco_layout_professional_detailed.png (con scores y IDs)")
        print(f"  → coco_original.png, coco_cropped.png")
        
        return layout, elementos_con_texto
        
    except Exception as e:
        print(f"❌ Error en análisis profesional COCO: {e}")
        import traceback
        traceback.print_exc()
        return lp.Layout(), []

print("✓ Función de análisis COCO profesional lista")

✓ Función de análisis COCO profesional lista


In [5]:
# EJECUCIÓN AUTOMÁTICA CON COCO
if MODEL_LOADED:
    print("🚀 INICIANDO ANÁLISIS AUTOMÁTICO CON COCO")
    print("=" * 60)
    
    # Buscar PDFs
    project_root = "/Users/alexa/Projects/cdmx_kg"
    search_paths = [
        os.path.join(project_root, "Mexico_City", "laws"),
        os.path.join(project_root, "Mexico_City", "laws_1"),
        project_root
    ]
    
    pdf_files = []
    for search_path in search_paths:
        if os.path.exists(search_path):
            for root, dirs, files in os.walk(search_path):
                for file in files:
                    if file.endswith('.pdf'):
                        pdf_files.append(os.path.join(root, file))
    
    if pdf_files:
        pdf_path = pdf_files[0]
        print(f"📄 Analizando con COCO: {os.path.basename(pdf_path)}")
        print(f"📍 Ruta: {pdf_path}")
        print()
        
        # Ejecutar análisis COCO profesional
        layout, elementos_con_texto = analyze_pdf_with_professional_coco(pdf_path)
        
        print(f"\n" + "="*70)
        print(f"🎯 COMPARACIÓN COCO vs SIMPLE:")
        print(f"\n✅ VENTAJAS DE COCO:")
        print(f"  • Clasificación semántica (Text, Title, List, Table, Figure)")
        print(f"  • Entrenado en millones de documentos")
        print(f"  • Entiende estructura del documento")
        print(f"  • No requiere ajuste manual de parámetros")
        print(f"  • Confianza por elemento")
        
        print(f"\n📊 RESULTADOS:")
        if elementos:
            print(f"  ✅ COCO detectó {len(elementos)} elementos estructurados")
            print(f"  📝 {len(elementos_con_texto)} contienen texto válido")
            print(f"  🎯 Próximo paso: Abrir coco_layout_detected.png")
            
            # Buscar artículos en títulos
            titulos = [e for e in elementos_con_texto if e.tipo == 'Title']
            articulos_encontrados = []
            for titulo in titulos:
                if re.search(r'art[íi]culo\s+\d+', titulo.texto, re.IGNORECASE):
                    articulos_encontrados.append(titulo)
            
            if articulos_encontrados:
                print(f"  🎯 ¡Encontrados {len(articulos_encontrados)} títulos de artículos!")
                for i, art in enumerate(articulos_encontrados[:3], 1):
                    preview = art.texto[:80] + '...' if len(art.texto) > 80 else art.texto
                    print(f"     {i}. \"{preview}\"")
        else:
            print(f"  ❌ COCO no detectó elementos")
            print(f"  💡 Posibles causas:")
            print(f"     • PDF es imagen escaneada (necesita OCR)")
            print(f"     • Umbral de confianza muy alto")
            print(f"     • Formato no estándar")
        
    else:
        print("❌ No se encontraron PDFs")
        
else:
    print("❌ MODELO COCO NO DISPONIBLE")
    print("🔄 Ejecutar celda 2 para cargar el modelo")
    print("💡 O instalar dependencias faltantes")

🚀 INICIANDO ANÁLISIS AUTOMÁTICO CON COCO
📄 Analizando con COCO: LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf
📍 Ruta: /Users/alexa/Projects/cdmx_kg/pdfs/LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf

🎯 ANÁLISIS PROFESIONAL COCO: LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf
✓ PDF: 25 páginas
✓ Imagen OpenCV: 1700x2200 (escala: 2.78)
✓ Original: ../data/debug/coco_original.png
📐 Recorte: 1700x2200 → 1600x1910
✓ Recortada: ../data/debug/coco_cropped.png

🎯 DETECCIÓN INTELIGENTE LAYOUTPARSER:
🎯 Detectando layout con layoutparser_smart...
  📐 Imagen: 1600x1910 pixels
  ✅ Detectados: 13 elementos
      Distribución: {'list': 13}
      Confianza promedio: 0.70 (min: 0.70, max: 0.70)

📊 ANÁLISIS ESTADÍSTICO DETALLADO:
  📋 LIST:
      Cantidad: 13
      Confianza promedio: 0.700
      Área promedio: 616 px²
      Área total: 8010 px²

🎨 CREANDO VISUALIZACIONES PROFESIONALES:
❌ Error en análisis profesional COCO: 'FreeTypeFont' object has no attribute 'getsize'

🎯 COMPARACIÓN COCO vs SIMPLE:

✅ VENTAJAS DE COCO:
  • Clasifi

Traceback (most recent call last):
  File "/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_52275/1628307153.py", line 57, in analyze_pdf_with_professional_coco
    viz_basic, viz_detailed = create_professional_visualization(cropped_cv, layout, viz_path)
  File "/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_52275/2345658601.py", line 106, in create_professional_visualization
    viz_detailed = lp.draw_box(cv_image, layout_with_ids,
  File "/opt/anaconda3/envs/cdmx_kg/lib/python3.10/site-packages/layoutparser/visualization.py", line 194, in wrap
    out = func(canvas, layout, *args, **kwargs)
  File "/opt/anaconda3/envs/cdmx_kg/lib/python3.10/site-packages/layoutparser/visualization.py", line 392, in draw_box
    text_w, text_h = font_obj.getsize(text)
AttributeError: 'FreeTypeFont' object has no attribute 'getsize'


NameError: name 'elementos' is not defined